In [1]:
import numpy as np
import pandas as pd
import glob
import re
from datetime import datetime
from sklearn.preprocessing import LabelEncoder


In [2]:
def is_top_model(x, models):
    if models.count(x)==0:
        x = "other"
    return x


## TEST DATA


In [3]:
# kaggle only:

'''
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

df_test = pd.read_csv('../input/sf-dst-car-price-prediction/test.csv')
df_train = pd.read_csv('../input/parsed-auto/train.csv')
df_sub = pd.read_csv('../input/sf-dst-car-price-prediction/sample_submission.csv')

'''

# local only:
df_test = pd.read_csv('test.csv')
df_sub = pd.read_csv('sample_submission.csv')


## COLLECT TRAIN DATA:


In [4]:
# collect data from files in CONTENT folder:
filenames = glob.glob("CONTENT/*content.csv")
lst_df_train = []
for filename in filenames:
    df_cur = pd.read_csv(filename, header=None)
    lst_df_train.append(df_cur)
df_train =  pd.concat(lst_df_train, axis=0, ignore_index=True)    

# get rid of duplicates and small check:   
df_train.drop_duplicates(inplace=True)
df_train.shape


(35265, 23)

## SAVE DF_TRAIN

In [5]:
# save to file:
df_train.to_csv("train.csv", index=False)


In [6]:
df_train.sample(3)


,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
2709,TOYOTA,Toyota RAV4 IV (CA40) Рестайлинг,внедорожник 5 дв.,5,2015,2017,белый,2.2 LTR,150 N12,дизель,...,ALLROAD_5_DOORS AUTOMATIC 2.2,автоматическая,1 владелец,Оригинал,полный,Левый,Не требует ремонта,Растаможен,https://auto.ru/cars/used/sale/toyota/rav_4/11...,1 997 000 ₽
444,TOYOTA,Toyota Land Cruiser 200 Series Рестайлинг 2,внедорожник 5 дв.,5,2015,2016,коричневый,4.5 LTR,249 N12,дизель,...,ALLROAD_5_DOORS AUTOMATIC 4.5,автоматическая,2 владельца,Оригинал,полный,Левый,Не требует ремонта,Растаможен,https://auto.ru/cars/used/sale/toyota/land_cru...,4 785 900 ₽
27567,MITSUBISHI,Mitsubishi Lancer IX Рестайлинг,универсал 5 дв.,5,2005,2008,серебристый,1.6 LTR,98 N12,бензин,...,WAGON_5_DOORS MECHANICAL 1.6,механическая,3 или более,Оригинал,передний,Левый,Не требует ремонта,Растаможен,https://auto.ru/cars/used/sale/mitsubishi/lanc...,320 000 ₽


In [7]:
df_test.sample(3)


,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,...,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня
6540,седан,VOLVO,https://auto.ru/cars/used/sale/volvo/s80/11012...,чёрный,"{""id"":""6337459"",""name"":""Summum"",""available_opt...",Автомобиль в идеальном техническом состоянии. ...,2.5 LTR,231 N12,"{""engine-proof"":true,""cruise-control"":true,""ti...",бензин,...,SEDAN AUTOMATIC 2.5,автоматическая,EUROPEAN,3 или более,1 год и 5 месяцев,Оригинал,передний,Левый,Не требует ремонта,Растаможен
31443,седан,VOLVO,https://auto.ru/cars/used/sale/volvo/760/11013...,красный,NaN,На ходу. Документы есть (с документами все в п...,2.8 LTR,156 N12,NaN,бензин,...,SEDAN MECHANICAL 2.8,механическая,EUROPEAN,3 или более,NaN,Дубликат,задний,Левый,Не требует ремонта,Растаможен
32094,седан,HONDA,https://auto.ru/cars/used/sale/honda/civic/110...,чёрный,NaN,"Добрый день! Продаю машину на отличном ходу, р...",1.5 LTR,90 N12,"{""tinted-glass"":true,""aux"":true,""usb"":true,""el...",бензин,...,SEDAN MECHANICAL 1.5,механическая,JAPANESE,2 владельца,NaN,Дубликат,передний,Левый,Не требует ремонта,Растаможен


In [8]:
df_sub.sample(3)


,sell_id,price
7718,1101265099,0
1252,1100884798,0
15655,1101316951,0


## COLUMNS:


- compare columns in train and test datasets, rename where necessary and drop from train if needed
- mark test and train data
- join datasets


In [9]:
# take a look at test columns:
test_columns = df_test.columns
test_columns


Index(['bodyType', 'brand', 'car_url', 'color', 'complectation_dict',
       'description', 'engineDisplacement', 'enginePower', 'equipment_dict',
       'fuelType', 'image', 'mileage', 'modelDate', 'model_info', 'model_name',
       'name', 'numberOfDoors', 'parsing_unixtime', 'priceCurrency',
       'productionDate', 'sell_id', 'super_gen', 'vehicleConfiguration',
       'vehicleTransmission', 'vendor', 'Владельцы', 'Владение', 'ПТС',
       'Привод', 'Руль', 'Состояние', 'Таможня'],
      dtype='object')

In [10]:
# take a look at train columns:
train_columns = df_train.columns
train_columns


Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22],
           dtype='int64')

In [11]:
# ooops... take another look at data to figure out how to name them:
df_train.head().T


,0,1,2,3,4
0,TOYOTA,TOYOTA,TOYOTA,TOYOTA,TOYOTA
1,Toyota Camry VIII (XV70),Toyota Camry VIII (XV70) Рестайлинг,Toyota Yaris I,"Toyota Corolla AMT X (E140, E150)",Toyota Land Cruiser Prado 90 Series Рестайлинг
2,седан,седан,хэтчбек 3 дв.,седан,внедорожник 5 дв.
3,4,4,3,4,5
4,2017,2020,1999,2006,1999
5,2020,2021,2000,2008,1999
6,чёрный,красный,синий,серый,серебристый
7,3.5 LTR,3.5 LTR,1.5 LTR,1.6 LTR,3.4 LTR
8,249 N12,249 N12,106 N12,124 N12,178 N12
9,бензин,бензин,бензин,бензин,бензин


In [12]:
# rename train columns:
train_columns = train_columns.map({0 : 'brand', 
                                   1 : 'model_name', 
                                   2 : 'bodyType',
                                   3 : 'numberOfDoors',
                                   4 : 'modelDate',
                                   5 : 'productionDate',
                                   6 : 'color',
                                   7 : 'engineDisplacement',
                                   8 : 'enginePower',
                                   9 : 'fuelType',
                                   10 : 'image',
                                   11 : 'mileage',
                                   12 : 'name',
                                   13 : 'vehicleConfiguration',
                                   14 : 'vehicleTransmission',
                                   15 : 'Владельцы',
                                   16 : 'ПТС',
                                   17 : 'Привод',
                                   18 : 'Руль',
                                   19 : 'Состояние',
                                   20 : 'Таможня', 
                                   21 : 'car_url',
                                   22 : 'price'})
df_train.columns = train_columns


In [13]:
# take a look if all good now:
df_train.sample().T


,29134
brand,MITSUBISHI
model_name,Mitsubishi ASX I Рестайлинг
bodyType,внедорожник 5 дв.
numberOfDoors,5
modelDate,2012
productionDate,2014
color,белый
engineDisplacement,1.6 LTR
enginePower,117 N12
fuelType,бензин


In [14]:
# find columns in test not being presented in train:
cols_to_drop_from_test = list(set(test_columns) - set(train_columns))
cols_to_drop_from_test


['vendor',
 'parsing_unixtime',
 'Владение',
 'complectation_dict',
 'equipment_dict',
 'description',
 'priceCurrency',
 'super_gen',
 'sell_id',
 'model_info']

In [15]:
# hmmmm ))) interesting, how old test dataset is:
datetime.fromtimestamp(df_test["parsing_unixtime"].min()).strftime('%Y-%b-%d'),\
datetime.fromtimestamp(df_test["parsing_unixtime"].max()).strftime('%Y-%b-%d')


('2020-Oct-19', '2020-Oct-26')

Данным ровно год. Цены выросли. По данным статистики, на автомобили - на 15% Возможно, следует промасштабировать цену...

In [16]:
# priceCurrency - hopefulyy, the same everywhere:
df_test.priceCurrency.value_counts()


RUB    34686
Name: priceCurrency, dtype: int64

OK, as in train dataset all prices are in RUB 

In [17]:
# check if we've got new columns during parsing:
new_columns = list(set(train_columns) - set(test_columns))
new_columns


['price']

that's ok, we are not supposed to know the price in test dataset


In [18]:
# compare main values in test and train to make sure we have good data:
df_test.brand.value_counts(), "==================", df_train.brand.value_counts()


(BMW           4473
 VOLKSWAGEN    4404
 NISSAN        4393
 MERCEDES      4180
 TOYOTA        3913
 AUDI          3421
 MITSUBISHI    2843
 SKODA         2741
 VOLVO         1463
 HONDA         1150
 INFINITI       871
 LEXUS          834
 Name: brand, dtype: int64,
 '==================',
 AUDI          3383
 SKODA         3383
 HONDA         3377
 BMW           3162
 NISSAN        3160
 MERCEDES      3083
 MITSUBISHI    3065
 TOYOTA        2992
 VOLKSWAGEN    2980
 LEXUS         2926
 VOLVO         2411
 INFINITI      1343
 Name: brand, dtype: int64)

There are only 12 brands represented in test dataset , for now therefore no need to process others

In [19]:
# join datasets marking them for future split:
df_test['price'] = 0
df_test['test_train'] = "test"
df_train['test_train'] = "train"
data = df_train.append(df_test, sort=False).reset_index(drop=True) # объединяем
data.shape


(69951, 34)

In [20]:
# remove columns we dot't have in train:
data.drop(cols_to_drop_from_test, axis=1, inplace=True)
data.shape


(69951, 24)

In [21]:
# rename columns to make their names more commfortable to use:
col_replacement = {'bodyType' : 'body_type', 
                   'numberOfDoors': 'doors',
                  'modelDate':'model_date',
                  'productionDate': 'prod_date',
                  'engineDisplacement':'engine_V',
                  'enginePower' : 'power',
                  'fuelType': 'fuel',
                  'vehicleConfiguration' : 'config',
                  'vehicleTransmission' : 'transmission',
                  'Владельцы' : 'owners',
                   'ПТС' : 'pass',
                   'Привод' : 'drive',
                   'Руль' : 'wheel',
                   'Состояние' : 'state',
                   'Таможня' : 'custom'
                  }
data.rename(columns=col_replacement, inplace=True)
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69951 entries, 0 to 69950
Data columns (total 24 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   brand         69951 non-null  object
 1   model_name    69951 non-null  object
 2   body_type     69951 non-null  object
 3   doors         69951 non-null  int64 
 4   model_date    69951 non-null  int64 
 5   prod_date     69951 non-null  int64 
 6   color         69951 non-null  object
 7   engine_V      69951 non-null  object
 8   power         69951 non-null  object
 9   fuel          69951 non-null  object
 10  image         69951 non-null  object
 11  mileage       69951 non-null  object
 12  name          69951 non-null  object
 13  config        69951 non-null  object
 14  transmission  69951 non-null  object
 15  owners        69951 non-null  object
 16  pass          69950 non-null  object
 17  drive         69951 non-null  object
 18  wheel         69951 non-null  object
 19  stat

## VALUE CLEANUP


- process columns manually one by one
- split column names into 3 lists: categorian, numeric and binary
- add new columns
- drop some columns


In [22]:
# lists to sort out columns for future processing:
cat_columns = []
num_columns = []
bin_columns = []


In [23]:
# when necesssary turn this on:
# pd.set_option('display.max_rows', None)


### 0. brand

In [24]:
# how many cars of each brand do we have:
data.brand.value_counts()


BMW           7635
NISSAN        7553
VOLKSWAGEN    7384
MERCEDES      7263
TOYOTA        6905
AUDI          6804
SKODA         6124
MITSUBISHI    5908
HONDA         4527
VOLVO         3874
LEXUS         3760
INFINITI      2214
Name: brand, dtype: int64

In [25]:
cat_columns.append("brand")


### 1. model_name

In [26]:
data.model_name.value_counts()


OCTAVIA                                    1418
5ER                                        1026
LANCER                                      855
E_KLASSE                                    809
PASSAT                                      782
                                           ... 
Nissan Largo II (GC22)                        1
BMW 5 серии 520i V (E60/E61) Рестайлинг       1
Toyota Town Ace II                            1
Toyota Sequoia I Рестайлинг                   1
SIENTA                                        1
Name: model_name, Length: 2861, dtype: int64

In [27]:
models_to_numbers = dict(data.model_name.value_counts())
models_to_numbers


{'OCTAVIA': 1418,
 '5ER': 1026,
 'LANCER': 855,
 'E_KLASSE': 809,
 'PASSAT': 782,
 'A6': 757,
 '3ER': 752,
 'POLO': 724,
 'CAMRY': 709,
 'X5': 675,
 'X_TRAIL': 654,
 'A4': 626,
 'QASHQAI': 609,
 'S_KLASSE': 549,
 'OUTLANDER': 544,
 'LAND_CRUISER': 514,
 'ALMERA': 502,
 'TIGUAN': 491,
 'Volkswagen Polo V Рестайлинг': 473,
 'TOUAREG': 469,
 'COROLLA': 456,
 'RAV_4': 447,
 'C_KLASSE': 428,
 'GOLF': 424,
 'Skoda Octavia II (A5) Рестайлинг': 400,
 'LAND_CRUISER_PRADO': 387,
 'TEANA': 374,
 'RAPID': 370,
 'Skoda Octavia III (A7) Рестайлинг': 361,
 'Skoda Octavia III (A7)': 356,
 '7ER': 343,
 'M_KLASSE': 343,
 'XC90': 340,
 'JETTA': 324,
 'X6': 317,
 'CIVIC': 316,
 'Volkswagen Tiguan I Рестайлинг': 311,
 'PAJERO': 306,
 'Nissan Qashqai II': 305,
 'Q7': 302,
 'FX': 297,
 'X3': 283,
 'Volvo XC90 I Рестайлинг': 277,
 'Nissan X-Trail III': 272,
 'JUKE': 268,
 'Volkswagen Tiguan II': 267,
 'Skoda Rapid I Рестайлинг': 262,
 'ACCORD': 260,
 'SUPERB': 248,
 'Mitsubishi Outlander III Рестайлинг 2': 24

In [28]:
top_models = []
for model in models_to_numbers:
    if models_to_numbers[model]>=30:
        top_models.append(model)
top_models


['OCTAVIA',
 '5ER',
 'LANCER',
 'E_KLASSE',
 'PASSAT',
 'A6',
 '3ER',
 'POLO',
 'CAMRY',
 'X5',
 'X_TRAIL',
 'A4',
 'QASHQAI',
 'S_KLASSE',
 'OUTLANDER',
 'LAND_CRUISER',
 'ALMERA',
 'TIGUAN',
 'Volkswagen Polo V Рестайлинг',
 'TOUAREG',
 'COROLLA',
 'RAV_4',
 'C_KLASSE',
 'GOLF',
 'Skoda Octavia II (A5) Рестайлинг',
 'LAND_CRUISER_PRADO',
 'TEANA',
 'RAPID',
 'Skoda Octavia III (A7) Рестайлинг',
 'Skoda Octavia III (A7)',
 '7ER',
 'M_KLASSE',
 'XC90',
 'JETTA',
 'X6',
 'CIVIC',
 'Volkswagen Tiguan I Рестайлинг',
 'PAJERO',
 'Nissan Qashqai II',
 'Q7',
 'FX',
 'X3',
 'Volvo XC90 I Рестайлинг',
 'Nissan X-Trail III',
 'JUKE',
 'Volkswagen Tiguan II',
 'Skoda Rapid I Рестайлинг',
 'ACCORD',
 'SUPERB',
 'Mitsubishi Outlander III Рестайлинг 2',
 'Q5',
 'S60',
 'FABIA',
 'Volkswagen Polo V',
 'Skoda Rapid I',
 'Skoda Fabia II Рестайлинг',
 'Mitsubishi Outlander III',
 'YETI',
 'MURANO',
 'CR_V',
 'RX',
 'GL_KLASSE',
 'A3',
 'Honda Civic VIII',
 'Skoda Kodiaq I',
 '1ER',
 'PRIMERA',
 'X1',
 

In [29]:
# reduce number of categories:
# if number of records for a model less than 30, replace model_name with "other" ...
data.model_name = data.model_name.apply(lambda x: is_top_model(x, top_models))


In [30]:
# sort out:
cat_columns.append("model_name")


### 2. body_type

In [31]:
data.body_type.value_counts()


внедорожник 5 дв.          27985
седан                      23726
лифтбек                     4919
хэтчбек 5 дв.               4270
универсал 5 дв.             2526
минивэн                     1684
купе                        1460
компактвэн                  1081
хэтчбек 3 дв.                747
пикап двойная кабина         551
купе-хардтоп                 228
внедорожник 3 дв.            198
кабриолет                    170
родстер                      152
фургон                       143
микровэн                      46
седан-хардтоп                 19
пикап одинарная кабина        14
седан 2 дв.                    8
лимузин                        8
пикап полуторная кабина        7
внедорожник открытый           6
тарга                          2
фастбек                        1
Name: body_type, dtype: int64

In [32]:
# translate body types:
data.body_type = data.body_type.map(
                {'седан' : 'sedan',
                 'внедорожник 5 дв.' : 'SUV 5 doors.',
                 'лифтбек' : 'liftback',
                 'хэтчбек 5 дв.' : 'hatchback 5 doors.',
                 'универсал 5 дв.' : 'station wagon 5 doors.',
                 'минивэн' : 'minivan',
                 'купе' : 'coupe',
                 'компактвэн' : 'compact MPV',
                 'хэтчбек 3 дв.' : 'hatchback 3 doors.',
                 'пикап двойная кабина' : 'pickup double cab',
                 'купе-хардтоп' : 'coupe-hardtop',
                 'внедорожник 3 дв.' : 'off-road vehicle 3 doors.',
                 'родстер' :  'roadster',
                 'фургон' : 'van',
                 'кабриолет' : 'cabriolet',
                 'седан-хардтоп' : 'hardtop sedan',
                 'микровэн' : 'microvan',
                 'седан 2 дв.' : 'sedan 2 doors.',
                 'лимузин' : 'limousine',
                 'пикап одинарная кабина' : 'pickup single cab',
                 'пикап полуторная кабина' : 'pickup one-and-a-half cab',
                 'внедорожник открытый' :  'off-road vehicle open',
                 'тарга' : 'targa',
                 'фастбек' : 'fastback'})


In [33]:
# sort out:
cat_columns.append("body_type")
cat_columns


['brand', 'model_name', 'body_type']

### 3. doors

In [34]:
data.doors.value_counts()


5    41563
4    25317
2     2060
3     1010
0        1
Name: doors, dtype: int64

In [35]:
# sort out:
cat_columns.append("doors")
cat_columns


['brand', 'model_name', 'body_type', 'doors']

### 4. model_date

In [36]:
data.model_date.describe()


count    69951.000000
mean      2008.272619
std          7.322240
min       1904.000000
25%       2005.000000
50%       2010.000000
75%       2013.000000
max       2021.000000
Name: model_date, dtype: float64

Dataset contains data about car model from year 1904... 

In [37]:
# sort out:
num_columns.append("model_date")
num_columns


['model_date']

In [38]:
# to add year of collecting datasets, find it out: 
# for train and test data it is different:
traindata_year = data[data.test_train=="train"]["model_date"].max()
testdata_year = data[data.test_train=="test"]["model_date"].max()
traindata_year, testdata_year


(2021, 2020)

### ++++++ new column +++++

In [39]:
# create new column "dataset_year" (year of collecting datasets)
data["dataset_year"] = np.where(data.test_train=="test", testdata_year, traindata_year)
data.dataset_year.sample(7)


58448    2020
35931    2020
7329     2021
43646    2020
40356    2020
18732    2021
63769    2020
Name: dataset_year, dtype: int64

In [40]:
# sort out:
bin_columns.append("dataset_year")
bin_columns


['dataset_year']

### ++++++ new column +++++

In [41]:
#### add model age:
data["model_age"]= data.dataset_year - data.model_date
data.model_age.sample(7)


11949     8
9590     13
21199     2
31978    12
54896    11
56815    23
22428     2
Name: model_age, dtype: int64

In [42]:
# sort out:
num_columns.append("model_age")
num_columns


['model_date', 'model_age']

### ++++++ new column +++++

In [43]:
# new column-marker to show if it is a new model on a market:
data["new_model"] = np.where(data.model_age==0, "yes", "no")


In [44]:
data.new_model.value_counts()


no     69927
yes       24
Name: new_model, dtype: int64

In [45]:
# sort out:
bin_columns.append("new_model")
bin_columns


['dataset_year', 'new_model']

### 5. prod_date

In [46]:
data.prod_date.describe()


count    69951.000000
mean      2010.393576
std          7.011943
min       1904.000000
25%       2007.000000
50%       2012.000000
75%       2016.000000
max       2021.000000
Name: prod_date, dtype: float64

In [47]:
# compare statistics for train and test datasets:
[data[data.test_train=="test"].prod_date.describe(), 
 "===============================",
 data[data.test_train=="train"].prod_date.describe()]



[count    34686.000000
 mean      2009.264602
 std          7.047661
 min       1904.000000
 25%       2006.000000
 50%       2011.000000
 75%       2014.000000
 max       2020.000000
 Name: prod_date, dtype: float64,
 '===============================',
 count    35265.000000
 mean      2011.504012
 std          6.796175
 min       1939.000000
 25%       2008.000000
 50%       2013.000000
 75%       2017.000000
 max       2021.000000
 Name: prod_date, dtype: float64]

test contains car produced in 1904, it will be no easy to predict  it's price on the base on train dataset...

In [48]:
# sort out:
num_columns.append("prod_date")
num_columns


['model_date', 'model_age', 'prod_date']

test dataset is one year older. Adding a car age column Add columns "raritet" , "new_model" and make feature categorian by 10 years.....

### ++++++++ new column +++++++++

In [49]:
data["car_age"] = data.dataset_year - data.model_date
data["car_age"].describe()


count    69951.000000
mean        12.231519
std          7.257905
min          0.000000
25%          7.000000
50%         11.000000
75%         15.000000
max        116.000000
Name: car_age, dtype: float64

There are new cars represented in both datasets, oldest car in test is much older then the oldest car in train, will be marked as raritet or even super raritet

In [50]:
# sort out:
num_columns.append("car_age")
num_columns


['model_date', 'model_age', 'prod_date', 'car_age']

In [51]:
# add new column "new_car" for cars with car_age=0:
data["new_car"] = np.where(data["car_age"]==0, "yes", "no")
# add new column "retro" for cars older then 30 years
data["retro"] = np.where(data["car_age"]>30, "yes", "no")
# and new column "raritet" for cars older then 50 years
data["raritet"] = np.where(data["car_age"]>50, "yes", "no")
data[data.car_age>30][["car_age", "retro", "raritet"]].sample(10)


,car_age,retro,raritet
18581,33,yes,no
3209,37,yes,no
40197,32,yes,no
1099,33,yes,no
21496,31,yes,no
40133,32,yes,no
40929,33,yes,no
40202,34,yes,no
8282,34,yes,no
42347,32,yes,no


In [52]:
# sort out:
bin_columns.append("new_car")
bin_columns.append("retro")
bin_columns.append("raritet")
bin_columns


['dataset_year', 'new_model', 'new_car', 'retro', 'raritet']

### 6. color

In [53]:
data.color.value_counts()


чёрный         21181
белый          14064
серый           8830
серебристый     7632
синий           6346
коричневый      3138
красный         3092
зелёный         1626
бежевый         1290
голубой          781
золотистый       490
пурпурный        482
фиолетовый       398
жёлтый           319
оранжевый        258
розовый           24
Name: color, dtype: int64

In [54]:
data.color = data.color.map(
                {"чёрный" : "black",
                "белый" : "white",
                "серый" : "grey",
                "серебристый" : "silver",
                "синий" : "blue",
                "красный" : "red",
                "коричневый" :"braun",
                "зелёный" : "green",
                "бежевый" : "beige",
                "голубой" : "light-blue",
                "золотистый" : "gold",
                "пурпурный" : "purple",
                "фиолетовый" : "violet",
                "жёлтый" : "yellow",
                "оранжевый" : "orange",
                "розовый" : "rose"})

data.color.value_counts()

black         21181
white         14064
grey           8830
silver         7632
blue           6346
braun          3138
red            3092
green          1626
beige          1290
light-blue      781
gold            490
purple          482
violet          398
yellow          319
orange          258
rose             24
Name: color, dtype: int64

In [55]:
# sort out:
cat_columns.append("color")
cat_columns


['brand', 'model_name', 'body_type', 'doors', 'color']

### 7. engine_V

In [56]:
data.engine_V.value_counts()


2.0 LTR    16022
1.6 LTR     9687
3.0 LTR     7501
1.8 LTR     5705
2.5 LTR     4708
2.4 LTR     3833
1.4 LTR     3506
3.5 LTR     3495
1.5 LTR     1957
1.2 LTR     1056
1.3 LTR      914
2.8 LTR      732
4.7 LTR      701
4.5 LTR      701
4.4 LTR      700
4.0 LTR      685
3.2 LTR      554
5.5 LTR      553
2.1 LTR      532
3.7 LTR      528
5.7 LTR      528
5.6 LTR      443
2.2 LTR      432
2.3 LTR      429
2.7 LTR      411
4.2 LTR      364
4.6 LTR      360
3.6 LTR      338
2.9 LTR      332
1.9 LTR      321
5.0 LTR      265
0.7 LTR      210
1.7 LTR      173
3.3 LTR      160
3.1 LTR      136
 LTR         127
4.8 LTR      110
1.0 LTR      105
3.8 LTR      101
4.3 LTR       84
2.6 LTR       80
6.0 LTR       76
3.4 LTR       69
4.1 LTR       54
5.4 LTR       45
6.2 LTR       40
5.2 LTR       23
5.9 LTR       15
6.3 LTR       13
6.6 LTR       13
5.8 LTR        9
1.1 LTR        9
4.9 LTR        4
5.3 LTR        1
3.9 LTR        1
Name: engine_V, dtype: int64

In [57]:
# remove "LTR"
data.engine_V = data.engine_V.apply(lambda x: str(x).replace(" LTR", ""))


In [58]:
# wonder what has happend with value "LTR":
data.engine_V.unique()


array(['3.5', '1.5', '1.6', '3.4', '1.8', '2.4', '4.5', '4.0', '2.8',
       '1.3', '2.5', '1.4', '2.0', '2.2', '3.0', '2.7', '4.2', '5.7',
       '3.3', '1.0', '4.7', '4.6', '4.3', '1.2', '3.6', '3.2', '1.9',
       '1.7', '2.3', '0.7', '4.4', '4.8', '6.6', '', '2.9', '6.0', '5.4',
       '5.0', '2.6', '5.9', '5.2', '3.1', '3.7', '6.3', '5.6', '3.8',
       '5.5', '2.1', '6.2', '5.8', '4.1', '1.1', '4.9', '3.9', '5.3'],
      dtype=object)

There is an empty value instead. Take a look:

In [59]:
data[data.engine_V==""]


,brand,model_name,body_type,doors,model_date,prod_date,color,engine_V,power,fuel,...,car_url,price,test_train,dataset_year,model_age,new_model,car_age,new_car,retro,raritet
6939,BMW,other,hatchback 5 doors.,5,2017,2018,black,,170 N12,электро,...,https://auto.ru/cars/used/sale/bmw/i3/11056537...,2 150 000 ₽,train,2021,4,no,4,no,no,no
7595,BMW,other,hatchback 5 doors.,5,2017,2019,beige,,170 N12,электро,...,https://auto.ru/cars/used/sale/bmw/i3/11056231...,2 500 000 ₽,train,2021,4,no,4,no,no,no
7722,BMW,other,hatchback 5 doors.,5,2013,2014,grey,,170 N12,электро,...,https://auto.ru/cars/used/sale/bmw/i3/11051492...,1 165 000 ₽,train,2021,8,no,8,no,no,no
7824,BMW,other,hatchback 5 doors.,5,2017,2018,blue,,170 N12,электро,...,https://auto.ru/cars/used/sale/bmw/i3/11042281...,2 495 000 ₽,train,2021,4,no,4,no,no,no
7842,BMW,other,hatchback 5 doors.,5,2017,2018,black,,170 N12,электро,...,https://auto.ru/cars/used/sale/bmw/i3/11016855...,1 920 000 ₽,train,2021,4,no,4,no,no,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67933,NISSAN,LEAF,hatchback 5 doors.,5,2017,2018,black,,150 N12,электро,...,https://auto.ru/cars/used/sale/nissan/leaf/110...,0,test,2020,3,no,3,no,no,no
67965,NISSAN,LEAF,hatchback 5 doors.,5,2010,2011,light-blue,,109 N12,электро,...,https://auto.ru/cars/used/sale/nissan/leaf/110...,0,test,2020,10,no,10,no,no,no
68063,NISSAN,LEAF,hatchback 5 doors.,5,2010,2013,black,,109 N12,электро,...,https://auto.ru/cars/used/sale/nissan/leaf/109...,0,test,2020,10,no,10,no,no,no
69424,MERCEDES,B_KLASSE,hatchback 5 doors.,5,2014,2016,white,,179 N12,электро,...,https://auto.ru/cars/used/sale/mercedes/b_klas...,0,test,2020,6,no,6,no,no,no


In [60]:
data.isnull().sum()


brand           0
model_name      0
body_type       0
doors           0
model_date      0
prod_date       0
color           0
engine_V        0
power           0
fuel            0
image           0
mileage         0
name            0
config          0
transmission    0
owners          0
pass            1
drive           0
wheel           0
state           0
custom          0
car_url         0
price           0
test_train      0
dataset_year    0
model_age       0
new_model       0
car_age         0
new_car         0
retro           0
raritet         0
dtype: int64

In [61]:
# sort out:
num_columns.append("engine_V")
num_columns


['model_date', 'model_age', 'prod_date', 'car_age', 'engine_V']

we received empty values for electrocars

In [62]:
### +++++++++++++ new column +++++++++++++++

In [63]:
# create marker column for electrocars:
data["electrocar"] = np.where(data["engine_V"]=="", "yes", "no")
data.electrocar.value_counts()


no     69824
yes      127
Name: electrocar, dtype: int64

In [64]:
# sort out:
bin_columns.append("electrocar")
bin_columns


['dataset_year', 'new_model', 'new_car', 'retro', 'raritet', 'electrocar']

### 8. power

In [65]:
data.power.value_counts()


150 N12    4104
249 N12    4060
110 N12    2557
170 N12    1898
140 N12    1845
           ... 
670 N12       1
430 N12       1
324 N12       1
338 N12       1
646 N12       1
Name: power, Length: 334, dtype: int64

In [66]:
# remove "N12":
data.power = data.power.apply(lambda x: str(x).replace(" N12", "")).astype(int)
data.power.value_counts()


150    4104
249    4060
110    2557
170    1898
140    1845
       ... 
236       1
301       1
430       1
324       1
44        1
Name: power, Length: 334, dtype: int64

In [67]:
# sort out
num_columns.append("power")
num_columns


['model_date', 'model_age', 'prod_date', 'car_age', 'engine_V', 'power']

In [68]:
# check for empty values:
data[data["power"]==""]


/Users/aliona/anaconda3/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


,brand,model_name,body_type,doors,model_date,prod_date,color,engine_V,power,fuel,...,price,test_train,dataset_year,model_age,new_model,car_age,new_car,retro,raritet,electrocar


In [69]:
data.power.sample(7)


26553     98
25026    150
11159    144
13506    150
7312     184
9976     114
48292    249
Name: power, dtype: int64

looks OK now

### 9. fuel

In [70]:
data.fuel.value_counts()


бензин     57409
дизель     11415
гибрид       992
электро      127
газ            8
Name: fuel, dtype: int64

In [71]:
data.fuel = data.fuel.map(
            {
                "бензин" : "petrol",
                "дизель" : "diesel",
                "гибрид" : "hybrid",
                "электро" : "electro",
                "газ" : "gas"
            }
                    )


In [72]:
data.fuel.value_counts()


petrol     57409
diesel     11415
hybrid       992
electro      127
gas            8
Name: fuel, dtype: int64

Too many petrol cars, too little of gas autos. 

### ++++++++++++ new column ++++++++++++

In [73]:
# Add column "petrol_car", as it's the most wanted
data["petrol_car"] = np.where(data.fuel=="petrol", "yes", "no")


In [74]:
# sort out
cat_columns.append("fuel")
bin_columns.append("petrol_car")
[cat_columns, bin_columns, num_columns]


[['brand', 'model_name', 'body_type', 'doors', 'color', 'fuel'],
 ['dataset_year',
  'new_model',
  'new_car',
  'retro',
  'raritet',
  'electrocar',
  'petrol_car'],
 ['model_date', 'model_age', 'prod_date', 'car_age', 'engine_V', 'power']]

### 10. image

In [75]:
data.image.sample(7)


42575    https://avatars.mds.yandex.net/get-autoru-vos/...
35627    https://autoru.naydex.net/ESooa9149/468b9eoMem...
15118    https://avatars.mds.yandex.net/get-autoru-vos/...
22862    https://autoru.naydex.net/Lr84PuH52/0ca95dIpb/...
39253    https://autoru.naydex.net/ys1kR7800/fbd964zfcB...
51111    https://autoru.naydex.net/ESooa9149/468b9eoMem...
50969    https://avatars.mds.yandex.net/get-autoru-vos/...
Name: image, dtype: object

In [76]:
# let's find out if there are empty or not valid values:
data[data.image.str[:6]!="https:"]


,brand,model_name,body_type,doors,model_date,prod_date,color,engine_V,power,fuel,...,test_train,dataset_year,model_age,new_model,car_age,new_car,retro,raritet,electrocar,petrol_car


In [77]:
data['image_len'] = data.image.astype(str).str.len()


In [78]:
data['image_len'].describe()


count    69951.000000
mean       156.511015
std         63.468004
min         94.000000
25%         94.000000
50%         95.000000
75%        221.000000
max        221.000000
Name: image_len, dtype: float64

In [79]:
data[data.image_len==94]["car_url"].sample()


21263    https://auto.ru/cars/used/sale/audi/q8/1105652...
Name: car_url, dtype: object

Looks like all ads have images, the colummn does not contain helpful infornation for feature analysis, will be removed 

### --------- remove column ---------

In [80]:
# remove column:
data.drop(["image"], axis=1, inplace=True)


### 11. mileage

In [81]:
data.mileage.head()


0     27 000 км
1     14 000 км
2    236 000 км
3    242 050 км
4    375 282 км
Name: mileage, dtype: object

In [82]:
data.mileage.sample(10)


24584    108 000 км
50454        200000
68223        238000
67634         95948
49345        110000
59552        137881
21632    172 000 км
25097      1 300 км
21730    130 000 км
22826    121 000 км
Name: mileage, dtype: object

In [83]:
data.mileage[0]


'27\xa0000\xa0км'

data is not clean, remove spaces and letters:

In [84]:
data.mileage = data.mileage.apply(
    lambda x: str(x).replace("\xa0", "").replace("км", "")
    ).astype(int)                                           


In [85]:
data.mileage.describe()


count      69951.000000
mean      153545.958843
std        98243.455772
min            1.000000
25%        82918.500000
50%       141000.000000
75%       205000.000000
max      1000000.000000
Name: mileage, dtype: float64

In [86]:
# sort out:
num_columns.append("mileage")
num_columns


['model_date',
 'model_age',
 'prod_date',
 'car_age',
 'engine_V',
 'power',
 'mileage']

### 12. name

In [87]:
data.name.sample(10)


9434                                 3.0 AT
26031                               1.6 AMT
10923                               2.0 CVT
20851                                2.5 MT
67885                2.5 CVT (169 л.с.) 4WD
68402            FX37 3.7 AT (333 л.с.) 4WD
28395                                2.0 AT
38004    60 TFSI Long 4.0 AT (460 л.с.) 4WD
41410                     1.8 MT (140 л.с.)
65869                2.5d AT (174 л.с.) 4WD
Name: name, dtype: object

In [88]:
len(data.name.value_counts())


2905

In [89]:
data[["engine_V", "transmission", "name", "power"]].sample(10)


,engine_V,transmission,name,power
2640,2.0,автоматическая,2.0 AT,130
60941,1.8,механическая,1.8 MT (90 л.с.),90
9031,2.5,механическая,2.5 MT,150
63969,1.5,автоматическая,1.5 AT (109 л.с.),109
65204,1.4,роботизированная,1.4 AMT (125 л.с.),125
31206,2.5,вариатор,2.5 CVT,152
20326,2.0,роботизированная,2.0 AMT,211
7319,3.0,автоматическая,3.0 AT,245
17842,2.0,автоматическая,2.0 AT,170
14640,1.6,роботизированная,1.6 AMT,122


Too many different values, column includes engine_V, transmission, power and drive. Can be deleted.

### --------- remove column ---------

In [90]:
data.drop(["name"], axis=1, inplace=True)


### 13. config


In [91]:
data.config.sample(10)


4672           WAGON_5_DOORS ROBOT 1.5
3800       HATCHBACK_5_DOORS ROBOT 1.8
30614    ALLROAD_5_DOORS AUTOMATIC 3.5
65465               SEDAN VARIATOR 3.1
24023          LIFTBACK MECHANICAL 1.6
12477     ALLROAD_5_DOORS VARIATOR 1.6
29052    ALLROAD_5_DOORS AUTOMATIC 2.4
29895    ALLROAD_5_DOORS AUTOMATIC 5.7
7946               SEDAN AUTOMATIC 2.0
21984                  SEDAN ROBOT 1.4
Name: config, dtype: object

contains other columns, will be removed

### --------- remove column ---------

In [92]:
data.drop(["config"], axis=1, inplace=True)


### 14. transmission

In [93]:
data.transmission.value_counts()


автоматическая      39203
механическая        13287
вариатор             9211
роботизированная     8250
Name: transmission, dtype: int64

In [94]:
# map values:
data.transmission = data.transmission.map(
            {"автоматическая" : "AT",
            "механическая" : "MT",
            "вариатор" : "CVT",
            "роботизированная" : "AMT"}
)

data.transmission.value_counts()


AT     39203
MT     13287
CVT     9211
AMT     8250
Name: transmission, dtype: int64

In [95]:
# sort out:
cat_columns.append("transmission")
cat_columns


['brand', 'model_name', 'body_type', 'doors', 'color', 'fuel', 'transmission']

### 15. owners

In [96]:
data.owners.value_counts()


3 или более    31221
1 владелец     21146
2 владельца    17584
Name: owners, dtype: int64

In [97]:
data.owners.value_counts().keys()[0],\
data.owners.value_counts().keys()[1],\
data.owners.value_counts().keys()[2]


('3 или более', '1\xa0владелец', '2\xa0владельца')

Column contains 3 values. As 1 owner is super good, add a flag to a car record: column "1_owner" 

In [98]:
dict_owners = {"3 или более":3, "1\xa0владелец": 1, "2\xa0владельца":2}
data.owners = data.owners.map(dict_owners)
data.owners.value_counts()


3    31221
1    21146
2    17584
Name: owners, dtype: int64

In [99]:
data["1_owner"] = np.where(data.owners==1, "yes", "no")


In [100]:
# sort out:
cat_columns.append("owners")
bin_columns.append("1_owner")
[cat_columns, bin_columns]


[['brand',
  'model_name',
  'body_type',
  'doors',
  'color',
  'fuel',
  'transmission',
  'owners'],
 ['dataset_year',
  'new_model',
  'new_car',
  'retro',
  'raritet',
  'electrocar',
  'petrol_car',
  '1_owner']]

### 16. pass


In [101]:
data["pass"].value_counts()


Оригинал    60310
Дубликат     9640
Name: pass, dtype: int64

column contains 2 values, convert it into a flag: 1 for original passport

In [102]:
data["pass"] = data["pass"].map({"Оригинал":"Original", "Дубликат":"Duplicate"})
data["pass"].value_counts()


Original     60310
Duplicate     9640
Name: pass, dtype: int64

In [103]:
# sort out:
bin_columns.append("pass")
bin_columns


['dataset_year',
 'new_model',
 'new_car',
 'retro',
 'raritet',
 'electrocar',
 'petrol_car',
 '1_owner',
 'pass']

### 17. drive

In [104]:
data["drive"].value_counts()


полный      32569
передний    30711
задний       6671
Name: drive, dtype: int64

not much of rare drive cars... not suitable for winter...  process as category feature:

In [105]:
data["drive"] = data["drive"].map({"передний":"FWD", "полный":"4WD", "задний":"RWD"})
data["drive"].value_counts()


4WD    32569
FWD    30711
RWD     6671
Name: drive, dtype: int64

In [106]:
cat_columns.append("drive")
cat_columns


['brand',
 'model_name',
 'body_type',
 'doors',
 'color',
 'fuel',
 'transmission',
 'owners',
 'drive']

### 18. wheel

In [107]:
data.wheel.value_counts()


Левый     65907
Правый     4044
Name: wheel, dtype: int64

binary feature, left steeering wheel will map to 1:

In [108]:
data["wheel"] = data["wheel"].map({"Левый": "Left", "Правый":"Right"})
data["wheel"].value_counts()


Left     65907
Right     4044
Name: wheel, dtype: int64

In [109]:
bin_columns.append("wheel")
bin_columns


['dataset_year',
 'new_model',
 'new_car',
 'retro',
 'raritet',
 'electrocar',
 'petrol_car',
 '1_owner',
 'pass',
 'wheel']

### 19. state

In [110]:
data.state.value_counts()


Не требует ремонта    69947
Битый / не на ходу        4
Name: state, dtype: int64

binary feature, the second value is too rare, but really important, it can 10 times reduce the price

### ------------------ remove column -------------------

In [111]:
# negative value for this column will significantly reduce price,
# but it is too rare be counted:
data.drop("state", axis=1, inplace=True)


### 20. custom

In [112]:
data.custom.value_counts()


Растаможен       69950
Не растаможен        1
Name: custom, dtype: int64

### ------------------ remove column -------------------

In [113]:
# negative value for this column will significantly reduce price,
# but it is too rare be counted:
data.drop("custom", axis=1, inplace=True)


### 21. car_url

not neede for analysis, delete:

In [114]:
data.drop("car_url", axis=1, inplace=True)


### 22. price

In [115]:
data.price.sample(10)


33962    3 350 000 ₽
64714              0
67039              0
62183              0
16133      520 000 ₽
42140              0
31720    1 915 000 ₽
46604              0
58698              0
22739      849 000 ₽
Name: price, dtype: object

contains 0-values, hopefully all from test dataset, let's take a look:

In [116]:
len(data[(data.price==0 )& (data.test_train=="train")]), len(df_test)


(0, 34686)

Train dataset doesn't contain 0 as a target value - OK

In [117]:
# take a deeper look at values:
data.price[0], data.price[1], data.price[3]


('2\xa0750\xa0000\xa0₽', '3\xa0250\xa0000\xa0₽', '650\xa0000\xa0₽')

Ooooopsss! Needs to be corrected

In [118]:
data.price = data.price.astype(str)
data.price = data["price"].apply(lambda x: x.replace("\xa0", "").replace("₽", ""))   
data.price = data.price.astype(int)
data.price.sample(10)


44836          0
11730    1050000
53302          0
47207          0
20638    1150000
42970          0
57490          0
20062     425000
64723          0
37552          0
Name: price, dtype: int64

In [119]:
data[data.price==0].test_train.value_counts()


test    34686
Name: test_train, dtype: int64

Price column looks OK now

### !!! For this column, will try to apply scaling, to take inflation into account

validate column lists:


In [120]:
all_columns = list(data.columns)
num_columns = list(set(num_columns))
cat_columns = list(set(cat_columns))
bin_columns = list(set(bin_columns))


In [121]:
print("num_columns:", num_columns)
print("cat_columns:", cat_columns)
print("bin_columns:", bin_columns)


num_columns: ['prod_date', 'model_date', 'model_age', 'engine_V', 'power', 'mileage', 'car_age']
cat_columns: ['body_type', 'transmission', 'brand', 'fuel', 'drive', 'doors', 'model_name', 'color', 'owners']
bin_columns: ['dataset_year', 'petrol_car', 'new_model', 'raritet', 'retro', 'electrocar', 'wheel', 'pass', '1_owner', 'new_car']


In [122]:
(set(num_columns) | set(cat_columns) | set(bin_columns)) - set(all_columns)


set()

In [123]:
set(all_columns) - (set(num_columns) | set(cat_columns) | set(bin_columns))


{'image_len', 'price', 'test_train'}

In [124]:
# we need price and test_train, remove image_len:
data.drop("image_len", axis=1, inplace=True)
# Columns "custom" and "state" "Not OK" values will significantly 
# reduce the price But, as not_OK values are too rare, columns will be removed




## SAVE DATA TO FILES:

In [125]:
# make sure data is OK to be saved:
data.head(7)#sample(7).T


,brand,model_name,body_type,doors,model_date,prod_date,color,engine_V,power,fuel,...,dataset_year,model_age,new_model,car_age,new_car,retro,raritet,electrocar,petrol_car,1_owner
0,TOYOTA,Toyota Camry VIII (XV70),sedan,4,2017,2020,black,3.5,249,petrol,...,2021,4,no,4,no,no,no,no,yes,yes
1,TOYOTA,other,sedan,4,2020,2021,red,3.5,249,petrol,...,2021,1,no,1,no,no,no,no,yes,yes
2,TOYOTA,other,hatchback 3 doors.,3,1999,2000,blue,1.5,106,petrol,...,2021,22,no,22,no,no,no,no,yes,no
3,TOYOTA,"Toyota Corolla AMT X (E140, E150)",sedan,4,2006,2008,grey,1.6,124,petrol,...,2021,15,no,15,no,no,no,no,yes,no
4,TOYOTA,other,SUV 5 doors.,5,1999,1999,silver,3.4,178,petrol,...,2021,22,no,22,no,no,no,no,yes,no
5,TOYOTA,Toyota Camry VIII (XV70),sedan,4,2017,2018,black,3.5,249,petrol,...,2021,4,no,4,no,no,no,no,yes,yes
6,TOYOTA,other,hatchback 3 doors.,3,2002,2003,black,1.8,192,petrol,...,2021,19,no,19,no,no,no,no,yes,no


In [126]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69951 entries, 0 to 69950
Data columns (total 28 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   brand         69951 non-null  object
 1   model_name    69951 non-null  object
 2   body_type     69951 non-null  object
 3   doors         69951 non-null  int64 
 4   model_date    69951 non-null  int64 
 5   prod_date     69951 non-null  int64 
 6   color         69951 non-null  object
 7   engine_V      69951 non-null  object
 8   power         69951 non-null  int64 
 9   fuel          69951 non-null  object
 10  mileage       69951 non-null  int64 
 11  transmission  69951 non-null  object
 12  owners        69951 non-null  int64 
 13  pass          69950 non-null  object
 14  drive         69951 non-null  object
 15  wheel         69951 non-null  object
 16  price         69951 non-null  int64 
 17  test_train    69951 non-null  object
 18  dataset_year  69951 non-null  int64 
 19  mode

In [127]:
data.to_csv("PROCESSED/data.csv", index=False)


In [128]:
df_num_columns = pd.DataFrame(num_columns)
df_cat_columns = pd.DataFrame(cat_columns)
df_bin_columns = pd.DataFrame(bin_columns)


df_num_columns.to_csv("PROCESSED/num_columns.csv", mode='w', header=False, index=False)
df_cat_columns.to_csv("PROCESSED/cat_columns.csv", mode='w', header=False, index=False)
df_bin_columns.to_csv("PROCESSED/bin_columns.csv", mode='w', header=False, index=False)


In [129]:
# quick check:
data = pd.read_csv("PROCESSED/data.csv")
data.head(7)#.T


,brand,model_name,body_type,doors,model_date,prod_date,color,engine_V,power,fuel,...,dataset_year,model_age,new_model,car_age,new_car,retro,raritet,electrocar,petrol_car,1_owner
0,TOYOTA,Toyota Camry VIII (XV70),sedan,4,2017,2020,black,3.5,249,petrol,...,2021,4,no,4,no,no,no,no,yes,yes
1,TOYOTA,other,sedan,4,2020,2021,red,3.5,249,petrol,...,2021,1,no,1,no,no,no,no,yes,yes
2,TOYOTA,other,hatchback 3 doors.,3,1999,2000,blue,1.5,106,petrol,...,2021,22,no,22,no,no,no,no,yes,no
3,TOYOTA,"Toyota Corolla AMT X (E140, E150)",sedan,4,2006,2008,grey,1.6,124,petrol,...,2021,15,no,15,no,no,no,no,yes,no
4,TOYOTA,other,SUV 5 doors.,5,1999,1999,silver,3.4,178,petrol,...,2021,22,no,22,no,no,no,no,yes,no
5,TOYOTA,Toyota Camry VIII (XV70),sedan,4,2017,2018,black,3.5,249,petrol,...,2021,4,no,4,no,no,no,no,yes,yes
6,TOYOTA,other,hatchback 3 doors.,3,2002,2003,black,1.8,192,petrol,...,2021,19,no,19,no,no,no,no,yes,no


In [130]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69951 entries, 0 to 69950
Data columns (total 28 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   brand         69951 non-null  object 
 1   model_name    69951 non-null  object 
 2   body_type     69951 non-null  object 
 3   doors         69951 non-null  int64  
 4   model_date    69951 non-null  int64  
 5   prod_date     69951 non-null  int64  
 6   color         69951 non-null  object 
 7   engine_V      69824 non-null  float64
 8   power         69951 non-null  int64  
 9   fuel          69951 non-null  object 
 10  mileage       69951 non-null  int64  
 11  transmission  69951 non-null  object 
 12  owners        69951 non-null  int64  
 13  pass          69950 non-null  object 
 14  drive         69951 non-null  object 
 15  wheel         69951 non-null  object 
 16  price         69951 non-null  int64  
 17  test_train    69951 non-null  object 
 18  dataset_year  69951 non-nu

Looks OK for future processing

### Please continue with 3. EDA